<a href="https://colab.research.google.com/github/DreamPearl/Q-Learning-for-Trading/blob/master/Datascrib_Q_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Clone the entire repo.
%cd /content/
!git clone -l -s https://github.com/DreamPearl/Q-Learning-for-Trading.git Datascrib-Q-Learning
%cd Datascrib-Q-Learning
!ls

/content
fatal: destination path 'Datascrib-Q-Learning' already exists and is not an empty directory.
/content/Datascrib-Q-Learning
agent.py		      requirements.txt
agent.pyc		      run.py
auquan_toolbox-2.1.92	      ta-lib
auquan_toolbox-2.1.92.tar.gz  ta-lib-0.4.0-src.tar.gz
data			      ta-lib-0.4.0-src.tar.gz.1
Datascrib_Q_Learning.ipynb    ta-lib-0.4.0-src.tar.gz.2
envs.py			      temp.txt
envs.pyc		      utils.py
model.py		      utils.pyc
model.pyc		      weights
portfolio_val		      yahooData
README.md


In [2]:
!git pull origin master

From https://github.com/DreamPearl/Q-Learning-for-Trading
 * branch            master     -> FETCH_HEAD
Already up to date.


In [3]:
!python2.7 -m pip install gym==0.9.4

In [4]:
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xzf ta-lib-0.4.0-src.tar.gz


--2020-12-09 14:47:10--  http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
Resolving prdownloads.sourceforge.net (prdownloads.sourceforge.net)... 216.105.38.13
Connecting to prdownloads.sourceforge.net (prdownloads.sourceforge.net)|216.105.38.13|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz [following]
--2020-12-09 14:47:10--  http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz
Resolving downloads.sourceforge.net (downloads.sourceforge.net)... 216.105.38.13
Reusing existing connection to prdownloads.sourceforge.net:80.
HTTP request sent, awaiting response... 302 Found
Location: https://versaweb.dl.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz [following]
--2020-12-09 14:47:10--  https://versaweb.dl.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz
Resolving versaweb.dl.so

In [5]:
%cd ta-lib/
!ls
!sudo ./configure
!sudo make
!sudo make install
!python2.7 -m pip install ta-lib
%cd ..

/content/Datascrib-Q-Learning/ta-lib
aclocal.m4	config.log     HISTORY.TXT  Makefile.am       ta-lib.dpkg
autogen.sh	config.status  include	    Makefile.in       ta-lib.dpkg.in
autom4te.cache	config.sub     install-sh   missing	      ta-lib.spec
bin		configure      libtool	    src		      ta-lib.spec.in
CHANGELOG.TXT	configure.in   ltmain.sh    ta-lib-config
config.guess	depcomp        Makefile     ta-lib-config.in
checking for a BSD-compatible install... /usr/bin/install -c
checking whether build environment is sane... yes
checking for a thread-safe mkdir -p... /bin/mkdir -p
checking for gawk... no
checking for mawk... mawk
checking whether make sets $(MAKE)... yes
checking for gcc... gcc
checking for C compiler default output file name... a.out
checking whether the C compiler works... yes
checking whether we are cross compiling... no
checking for suffix of executables... 
checking for suffix of object files... o
checking whether we are using the GNU C compiler... yes
checking whether 

In [6]:
!python2.7 -m pip install ta-lib
%env LD_LIBRARY_PATH=/usr/local/lib:$LD_LIBRARY_PATH

env: LD_LIBRARY_PATH=/usr/local/lib:$LD_LIBRARY_PATH


In [7]:
!sudo apt-get install python-tk
!python2.7 -m pip install --upgrade setuptools
!python2.7 -m pip install tensorboardX            #This must be installed before auquan_toolbox
!python2.7 -m pip install auquan_toolbox 
# !wget https://files.pythonhosted.org/packages/d7/30/65e90e69beaeb2a266759ce8d90378a8edcc86f81c0ba451ee78ad33cd87/auquan_toolbox-2.1.92.tar.gz
# !tar -xzf auquan_toolbox-2.1.92.tar.gz
# %cd auquan_toolbox-2.1.92
# !ls -l

# !python2.7 setup.py install
# %cd ..

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-tk is already the newest version (2.7.17-1~18.04).
0 upgraded, 0 newly installed, 0 to remove and 14 not upgraded.
Requirement already up-to-date: setuptools in /usr/local/lib/python2.7/dist-packages (44.1.1)


In [58]:
!python2.7 run.py --mode train -e 1 -b 50 -c AAPL

Using TensorFlow backend.
Processing data for stock: AAPL
20% done...
40% done...
60% done...
80% done...
           Trade_Date day_of_week        RSI       Chaikin  Adj_Close
2015-12-01 2015-12-01     Tuesday        NaN           NaN  26.957327
2015-12-02 2015-12-02   Wednesday        NaN           NaN  26.713804
2015-12-03 2015-12-03    Thursday        NaN           NaN  26.465693
2015-12-04 2015-12-04      Friday        NaN           NaN  27.345581
2015-12-07 2015-12-07      Monday        NaN           NaN  27.173285
2015-12-08 2015-12-08     Tuesday        NaN           NaN  27.161795
2015-12-09 2015-12-09   Wednesday        NaN           NaN  26.562178
2015-12-10 2015-12-10    Thursday        NaN           NaN  26.688534
2015-12-11 2015-12-11      Friday        NaN           NaN  26.001623
2015-12-14 2015-12-14      Monday        NaN -2.816928e+06  25.840809
2015-12-15 2015-12-15     Tuesday        NaN -3.987429e+07  25.383627
2015-12-16 2015-12-16   Wednesday        NaN -9.002003

In [59]:
!date
!ls -l weights/
!ls -l weights/ | tail -n 1 | awk '{print $9}'


Wed Dec  9 16:49:34 UTC 2020
total 4484
-rw-r--r-- 1 root root 237600 Dec  9 14:42 202012091441-dqn.h5
-rw-r--r-- 1 root root 237600 Dec  9 14:46 202012091446-dqn.h5
-rw-r--r-- 1 root root 237600 Dec  9 14:49 202012091447-dqn.h5
-rw-r--r-- 1 root root 237600 Dec  9 14:53 202012091452-dqn.h5
-rw-r--r-- 1 root root 237600 Dec  9 14:56 202012091455-dqn.h5
-rw-r--r-- 1 root root 237600 Dec  9 15:07 202012091457-dqn.h5
-rw-r--r-- 1 root root 237600 Dec  9 15:17 202012091511-dqn.h5
-rw-r--r-- 1 root root 237600 Dec  9 15:36 202012091528-dqn.h5
-rw-r--r-- 1 root root 237600 Dec  9 16:05 202012091602-dqn.h5
-rw-r--r-- 1 root root 237600 Dec  9 16:09 202012091608-dqn.h5
-rw-r--r-- 1 root root 237600 Dec  9 16:12 202012091610-dqn.h5
-rw-r--r-- 1 root root 237600 Dec  9 16:33 202012091632-dqn.h5
-rw-r--r-- 1 root root 237600 Dec  9 16:36 202012091635-dqn.h5
-rw-r--r-- 1 root root 237600 Dec  9 16:40 202012091637-dqn.h5
-rw-r--r-- 1 root root 237600 Dec  9 16:42 202012091641-dqn.h5
-rw-r--r-- 1 ro

In [60]:
!python2.7 run.py -m test -e 1 -c AAPL -w weights/`ls -l weights/ | tail -n 1 | awk '{print $9}'` # 202012091457-dqn.h5

Using TensorFlow backend.
Processing data for stock: AAPL
20% done...
40% done...
60% done...
80% done...
           Trade_Date day_of_week        RSI       Chaikin  Adj_Close
2015-12-01 2015-12-01     Tuesday        NaN           NaN  26.957327
2015-12-02 2015-12-02   Wednesday        NaN           NaN  26.713804
2015-12-03 2015-12-03    Thursday        NaN           NaN  26.465693
2015-12-04 2015-12-04      Friday        NaN           NaN  27.345581
2015-12-07 2015-12-07      Monday        NaN           NaN  27.173285
2015-12-08 2015-12-08     Tuesday        NaN           NaN  27.161795
2015-12-09 2015-12-09   Wednesday        NaN           NaN  26.562178
2015-12-10 2015-12-10    Thursday        NaN           NaN  26.688534
2015-12-11 2015-12-11      Friday        NaN           NaN  26.001623
2015-12-14 2015-12-14      Monday        NaN -2.816928e+06  25.840809
2015-12-15 2015-12-15     Tuesday        NaN -3.987429e+07  25.383627
2015-12-16 2015-12-16   Wednesday        NaN -9.002003